## Analysis
- Seasons of interest - June, July, August, September.
- Districts of interest - Kolhapur, Latur

- A new LSTM model to predict rainfall.

### Import libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.patches as mpatches 
from matplotlib.collections import PatchCollection
import plotly.figure_factory as ff
from IPython.display import HTML, display
from IPython.core import display as ICD
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import Artificial_Neural_Networks as ANN
import ARIMA

import math
from itertools import groupby
%matplotlib inline
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import plot_model
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential
from keras.layers.merge import concatenate
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import model_from_json

from importlib import reload
import itertools

### Useful functions

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

def calculate_performance(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    return round(mse, 3), round(mae, 3), round(mape, 3), round(rmse, 3)

### Dataset

In [4]:
PATH = 'Dataset/rainfall_data_1901_to_2002.xlsx'
data = pd.read_excel(PATH)

### Preprocess data

In [5]:
data = data.drop(columns='vlookup')
data = data[data['Year'].notnull()]
data['Year'] = data.Year.astype('int')
data.index = range(len(data))

m_data = data[data['State'] == 'Maharashtra']
m_data = m_data.drop(columns='State')

districts = m_data.District.unique()
years = list(range(1901, 2003))
months = data.columns[3:]
year_month = [str(year) + '_' + month for year in years for month in months]
dates = pd.date_range(start='1901-01', freq='MS', periods=len(years)*12)

maharashtra_data = pd.DataFrame({'Year_Month': year_month})
maharashtra_data['Date'] = dates
maharashtra_data[['Year', 'Month']] = maharashtra_data['Year_Month'].str.split('_', n=1, expand=True)
maharashtra_data = maharashtra_data.drop(columns=['Year_Month'])

for district in districts:
    df = m_data[m_data.District == district].drop(columns=['District', 'Year'])
    df = df.as_matrix().reshape((len(years) * len(months), 1))[:,0]
    maharashtra_data[district] = df

maharashtra_data.head()

,Date,Year,Month,Ahmadnagar,Akola,Amravati,Aurangabad,Bhandara,Bid,Buldana,...,Nashik,Osmanabad,Parbhani,Pune,Sangli,Satara,Solapur,Wardha,Washim,Yavatmal
0,1901-01-01,1901,Jan,2.510,34.202,35.651,10.922,23.397,16.647,31.455,...,5.063,15.500,33.207,0.922,0.138,0.197,4.496,45.050,32.485,22.031
1,1901-02-01,1901,Feb,11.489,1.099,6.822,4.362,63.844,1.916,0.823,...,1.609,2.784,4.997,7.195,0.537,0.525,3.667,28.220,3.607,16.198
2,1901-03-01,1901,Mar,11.325,30.002,36.103,25.161,33.563,27.287,28.448,...,11.196,11.333,31.625,5.105,13.090,9.566,10.330,26.398,26.557,29.318
3,1901-04-01,1901,Apr,33.931,10.248,10.636,12.714,61.560,33.211,13.902,...,7.838,34.814,41.941,35.949,50.077,30.110,59.854,29.228,18.127,30.521
4,1901-05-01,1901,May,30.401,2.891,4.173,34.244,13.665,59.027,9.397,...,7.475,52.792,31.794,36.650,78.994,65.226,50.892,8.619,6.700,13.193


In [6]:
m_data = maharashtra_data.copy()

In [7]:
m_data.head()

,Date,Year,Month,Ahmadnagar,Akola,Amravati,Aurangabad,Bhandara,Bid,Buldana,...,Nashik,Osmanabad,Parbhani,Pune,Sangli,Satara,Solapur,Wardha,Washim,Yavatmal
0,1901-01-01,1901,Jan,2.510,34.202,35.651,10.922,23.397,16.647,31.455,...,5.063,15.500,33.207,0.922,0.138,0.197,4.496,45.050,32.485,22.031
1,1901-02-01,1901,Feb,11.489,1.099,6.822,4.362,63.844,1.916,0.823,...,1.609,2.784,4.997,7.195,0.537,0.525,3.667,28.220,3.607,16.198
2,1901-03-01,1901,Mar,11.325,30.002,36.103,25.161,33.563,27.287,28.448,...,11.196,11.333,31.625,5.105,13.090,9.566,10.330,26.398,26.557,29.318
3,1901-04-01,1901,Apr,33.931,10.248,10.636,12.714,61.560,33.211,13.902,...,7.838,34.814,41.941,35.949,50.077,30.110,59.854,29.228,18.127,30.521
4,1901-05-01,1901,May,30.401,2.891,4.173,34.244,13.665,59.027,9.397,...,7.475,52.792,31.794,36.650,78.994,65.226,50.892,8.619,6.700,13.193


### Useful functions

In [8]:
def get_latest_file(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

In [9]:
def LSTM_model(num_of_previous_months, hidden_nodes_months, 
               num_of_previous_years, hidden_nodes_years, output_nodes):
    
    visible1 = Input((num_of_previous_months, 1))
#     visible1 = Input((1, num_of_previous_months))
    extract1 = LSTM(hidden_nodes_months, activation='relu')(visible1)

    visible2 = Input((num_of_previous_years, 1))
#     visible2 = Input((1, num_of_previous_years))
    extract2 = LSTM(hidden_nodes_years, activation='relu')(visible2)

    merge = concatenate([extract1, extract2])
    output = Dense(output_nodes)(merge)
    
    model = Model(inputs = [visible1, visible2], outputs = output)
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    plot_model(model, 'Functional_LSTM.png', show_shapes=True, show_layer_names=True)
    
    return model

In [10]:
def preprocess_data(m_data, district, month, num_of_prev_months, num_of_prev_years):
    
#     rainfall_season_data = m_data[['Date', 'Year', 'Month'] + districts_of_interest]
    rainfall_data = m_data[['Date', 'Year', 'Month', district]]
    month_data = rainfall_data[rainfall_data.Month == month]
    
    start_year = int(rainfall_data.Year.min())
    last_year = int(rainfall_data.Year.max())
    current_year = start_year + num_of_previous_years
    month_data_index = month_data.index
    
    train_data_input_1 = []
    for index in month_data_index[num_of_previous_years:]:
        data = list(rainfall_data.iloc[index - num_of_previous_months:index][district])
        train_data_input_1.append(data)
    train_data_input_1 = np.array(train_data_input_1)
    shape = train_data_input_1.shape
    train_data_input_1 = train_data_input_1.reshape(shape[0], shape[1], 1)
#     train_data_input_1 = train_data_input_1.reshape(shape[0], 1, shape[1])
    
    month_data_prep = list(month_data[district])
    train_data_input_2 = []
    for i in range(0, len(month_data_prep) - num_of_previous_years):
        data = month_data_prep[i:i+num_of_previous_years]
        train_data_input_2.append(data)
    train_data_input_2 = np.array(train_data_input_2)
    shape = train_data_input_2.shape
    train_data_input_2 = train_data_input_2.reshape(shape[0], shape[1], 1)
#     train_data_input_2 = train_data_input_2.reshape(shape[0], 1, shape[1])
    
    y_train = list(month_data.iloc[num_of_previous_years:][district])
    y_train = np.array(y_train)
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    
    return train_data_input_1, train_data_input_2, y_train

In [20]:
def split_and_train_LSTM(model, input_1, input_2, y_train_main, future_steps, epochs, batch_size):
    X_train_input_1, X_test_input_1, y_train, y_test = train_test_split(input_1, y_train_main, 
                                                                    test_size=future_steps, random_state=42)
    X_train_input_2, X_test_input_2, y_train, y_test = train_test_split(input_2, y_train_main, 
                                                                    test_size=future_steps, random_state=42)
    
    SOURCE_PATH = 'Models/'
    checkpoint = ModelCheckpoint(SOURCE_PATH + 'model-{epoch:03d}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  
    
    model.fit([X_train_input_1, X_train_input_2], y_train, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True, validation_split=0.1, callbacks=[checkpoint])
    
    file_name = get_latest_file(SOURCE_PATH)
    model.load_weights(file_name)
#     os.system('rm -rf %s/*' % SOURCE_PATH)
    
    return model, X_test_input_1, X_test_input_2, y_test

In [21]:
def predict_LSTM(model, X_test_input_1, X_test_input_2):
    y_pred = model.predict([X_test_input_1, X_test_input_2])
    return y_pred

In [22]:
def Long_Short_Term_Memory(data, district, month, num_of_prev_months, num_of_prev_years, hidden_nodes_months, hidden_nodes_years, epochs, batch_size, future_steps):
    model = LSTM_model(num_of_prev_months, hidden_nodes_months, num_of_prev_years, hidden_nodes_years, output_nodes)
    train_data_input_1, train_data_input_2, y_train_main = preprocess_data(m_data, district, month, num_of_prev_months, num_of_prev_years)
    model, X_test_input_1, X_test_input_2, y_test = split_and_train_LSTM(model, train_data_input_1, train_data_input_2, y_train_main, future_steps, epochs, batch_size)
    
    y_pred = predict_LSTM(model, X_test_input_1, X_test_input_2)
    return model, y_pred

In [23]:
future_steps = 5

# number_of_previous_months, number_of_previous_years, hidden_nodes_months, hidden_nodes_years, epochs, batch_size, future_steps
parameters_LSTM = [[3,6,9,12], [4,6,8], [6,8,10,12], [5,7,8], [100], [10], [future_steps]]
num_of_previous_months = 12
num_of_previous_years = 7
hidden_nodes_months = 12
hidden_nodes_years = 5
output_nodes = 1

In [24]:
model, y_pred = Long_Short_Term_Memory(m_data, district, month, num_of_previous_months, num_of_previous_years,
                      hidden_nodes_months, hidden_nodes_years, 250, 10, future_steps)

Train on 81 samples, validate on 9 samples
Epoch 1/250
81/81 [==============================] - 2s 20ms/step - loss: 395320.1066 - val_loss: 496618.3750

Epoch 00001: val_loss improved from inf to 496618.37500, saving model to Models/model-001.h5
Epoch 2/250
81/81 [==============================] - 0s 1ms/step - loss: 394541.9336 - val_loss: 495488.8438

Epoch 00002: val_loss improved from 496618.37500 to 495488.84375, saving model to Models/model-002.h5
Epoch 3/250
81/81 [==============================] - 0s 1ms/step - loss: 391736.0149 - val_loss: 490809.7188

Epoch 00003: val_loss improved from 495488.84375 to 490809.71875, saving model to Models/model-003.h5
Epoch 4/250
81/81 [==============================] - 0s 2ms/step - loss: 379145.1034 - val_loss: 462196.3438

Epoch 00004: val_loss improved from 490809.71875 to 462196.34375, saving model to Models/model-004.h5
Epoch 5/250
81/81 [==============================] - 0s 2ms/step - loss: 354466.6489 - val_loss: 420436.7500

Epoch 0

81/81 [==============================] - 0s 2ms/step - loss: 58564.6794 - val_loss: 50890.8281

Epoch 00046: val_loss did not improve from 34334.44922
Epoch 47/250
81/81 [==============================] - 0s 2ms/step - loss: 58427.5617 - val_loss: 51374.6328

Epoch 00047: val_loss did not improve from 34334.44922
Epoch 48/250
81/81 [==============================] - 0s 1ms/step - loss: 58173.5552 - val_loss: 52582.2852

Epoch 00048: val_loss did not improve from 34334.44922
Epoch 49/250
81/81 [==============================] - 0s 2ms/step - loss: 58103.9003 - val_loss: 51610.6289

Epoch 00049: val_loss did not improve from 34334.44922
Epoch 50/250
81/81 [==============================] - 0s 2ms/step - loss: 57854.0184 - val_loss: 51343.7461

Epoch 00050: val_loss did not improve from 34334.44922
Epoch 51/250
81/81 [==============================] - 0s 2ms/step - loss: 57722.2927 - val_loss: 50273.5820

Epoch 00051: val_loss did not improve from 34334.44922
Epoch 52/250
81/81 [=========

81/81 [==============================] - 0s 1ms/step - loss: 56407.3012 - val_loss: 66161.9453

Epoch 00096: val_loss did not improve from 34334.44922
Epoch 97/250
81/81 [==============================] - 0s 1ms/step - loss: 57011.0873 - val_loss: 69021.3047

Epoch 00097: val_loss did not improve from 34334.44922
Epoch 98/250
81/81 [==============================] - 0s 1ms/step - loss: 55994.4363 - val_loss: 71649.9609

Epoch 00098: val_loss did not improve from 34334.44922
Epoch 99/250
81/81 [==============================] - 0s 991us/step - loss: 55118.4288 - val_loss: 71962.8906

Epoch 00099: val_loss did not improve from 34334.44922
Epoch 100/250
81/81 [==============================] - 0s 1ms/step - loss: 54714.3690 - val_loss: 72771.5078

Epoch 00100: val_loss did not improve from 34334.44922
Epoch 101/250
81/81 [==============================] - 0s 1ms/step - loss: 54498.1034 - val_loss: 72362.5312

Epoch 00101: val_loss did not improve from 34334.44922
Epoch 102/250
81/81 [====

81/81 [==============================] - 0s 1ms/step - loss: 44132.8099 - val_loss: 52337.5469

Epoch 00146: val_loss did not improve from 34334.44922
Epoch 147/250
81/81 [==============================] - 0s 1ms/step - loss: 43946.7181 - val_loss: 53121.2773

Epoch 00147: val_loss did not improve from 34334.44922
Epoch 148/250
81/81 [==============================] - 0s 1ms/step - loss: 43945.2957 - val_loss: 53685.8906

Epoch 00148: val_loss did not improve from 34334.44922
Epoch 149/250
81/81 [==============================] - 0s 1ms/step - loss: 44224.6701 - val_loss: 52874.7031

Epoch 00149: val_loss did not improve from 34334.44922
Epoch 150/250
81/81 [==============================] - 0s 1ms/step - loss: 50623.2766 - val_loss: 52069.4727

Epoch 00150: val_loss did not improve from 34334.44922
Epoch 151/250
81/81 [==============================] - 0s 1ms/step - loss: 52113.9209 - val_loss: 51747.1094

Epoch 00151: val_loss did not improve from 34334.44922
Epoch 152/250
81/81 [===

81/81 [==============================] - 0s 1ms/step - loss: 45613.0606 - val_loss: 48133.3477

Epoch 00196: val_loss did not improve from 34334.44922
Epoch 197/250
81/81 [==============================] - 0s 1ms/step - loss: 45959.1534 - val_loss: 47701.5000

Epoch 00197: val_loss did not improve from 34334.44922
Epoch 198/250
81/81 [==============================] - 0s 1ms/step - loss: 46207.8269 - val_loss: 47477.9219

Epoch 00198: val_loss did not improve from 34334.44922
Epoch 199/250
81/81 [==============================] - 0s 1ms/step - loss: 45875.9316 - val_loss: 47464.1250

Epoch 00199: val_loss did not improve from 34334.44922
Epoch 200/250
81/81 [==============================] - 0s 1ms/step - loss: 45227.7488 - val_loss: 47347.9570

Epoch 00200: val_loss did not improve from 34334.44922
Epoch 201/250
81/81 [==============================] - 0s 1ms/step - loss: 44981.6322 - val_loss: 47204.2852

Epoch 00201: val_loss did not improve from 34334.44922
Epoch 202/250
81/81 [===

81/81 [==============================] - 0s 1ms/step - loss: 42756.0942 - val_loss: 43084.9219

Epoch 00246: val_loss did not improve from 34334.44922
Epoch 247/250
81/81 [==============================] - 0s 1ms/step - loss: 43322.6959 - val_loss: 42902.0703

Epoch 00247: val_loss did not improve from 34334.44922
Epoch 248/250
81/81 [==============================] - 0s 1ms/step - loss: 45699.3289 - val_loss: 42715.6328

Epoch 00248: val_loss did not improve from 34334.44922
Epoch 249/250
81/81 [==============================] - 0s 1ms/step - loss: 44133.2085 - val_loss: 41833.0352

Epoch 00249: val_loss did not improve from 34334.44922
Epoch 250/250
81/81 [==============================] - 0s 1ms/step - loss: 44241.7742 - val_loss: 37730.5156

Epoch 00250: val_loss did not improve from 34334.44922


In [ ]:
districts_of_interest = ['Kolhapur', 'Latur']
months_of_interest = ['Jun', 'Jul', 'Aug', 'Sep']

In [18]:
district = 'Kolhapur'
month = 'Jun'

In [ ]:
month = 'Jun'
preprocess_data(rainfall_data, month, num_of_previous_months, num_of_previous_years)

In [ ]:
# for district in districts_of_interest:
#     temp_data = rainfall_season_data[['Date', 'Year', 'Month', district]]
#     for month in months_of_interest:
#         df = temp_data[temp_data.Month == month]

In [ ]:
# LSTM_model(num_of_previous_months, hidden_nodes_months, num_of_previous_years, hidden_nodes_years, output_nodes)